In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome()

In [3]:
def search_with_keyword(keyword):
    web = 'https://www.amazon.com'
    driver.get(web)
    search = driver.find_element(By.ID, 'twotabsearchtextbox')
    search.send_keys(keyword)
    search_button = driver.find_element(By.ID, 'nav-search-submit-button')
    search_button.click()

In [4]:
search_with_keyword("wireless charger")

In [5]:
def navigate_next_page(search_term):
    template = "https://www.amazon.com/s?k={}&crid=5BUUOR41Y8NW&sprefix=wireless+charger%2Caps%2C199&ref=nb_sb_noss_1"
    seach_term = search_term.replace(' ', '+')
    link = template.format(search_term)
    link += '&page{}'
    
    return link

In [6]:
product_asin = []
product_name = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

soup = BeautifulSoup(driver.page_source, 'html.parser')


for page in(1, 21):
    driver.get(navigate_next_page("wireless charger").format(page))
    items = soup.find_all('div', {'data-component-type' : 's-search-result'})
    
    for item in items:
        # find name
        atag = item.h2.a
        name = atag.text.strip()
        product_name.append(name)

        # find ASIN number 
        data_asin = item.find('div', "data-asin")
        product_asin.append(data_asin)

        # find price
        try:
            price = item.find('span', 'a-price').find('span', 'a-offscreen').text

        except AttributeError:
            price = 0
        product_price.append(price)

        # find ratings and ratings_num
        try:
            rating = item.i.text
            ratings_num = item.find('span', {'class' : 'a-size-base s-underline-text'}).text.replace('(','').replace(')','')
        except AttributeError:
            rating = 0
            ratings_num = 0

        product_ratings.append(rating)
        product_ratings_num.append(str(ratings_num))

        # find link
        try:
            link = "https://www.amazon.com" + atag.get('href')
        except:
            link = ''
        product_link.append(link)

driver.quit()

# to check data scraped
print(product_name)
print(product_asin)
print(product_price)
print(product_ratings)
print(product_ratings_num)
print(product_link)

['Anker 315 Wireless Charger (Pad), 10W Max Fast Charging, Compatible with iPhone 14/13 Series, Samsung S22, AirPods, Samsung Buds, Google Buds, and More (Wall Charger Not Included)', 'Yootech Wireless Charger,10W Max Fast Wireless Charging Pad Compatible with iPhone 14/14 Plus/14 Pro/14 Pro Max/13/13 Mini/SE 2022/12/11/X/8,Samsung Galaxy S22/S21/S20,AirPods Pro 2(No AC Adapter)', 'Anker 313 Wireless Charger (Stand), Qi-Certified for iPhone 14/14 Pro/14 Pro Max/13/13 Pro Max, 10W Fast-Charging Galaxy S20, S10 (No AC Adapter)', 'Apple MagSafe Charger - Wireless Charger with Fast Charging Capability, Type C Wall Charger, Compatible with iPhone and AirPods', 'INIU Wireless Charger, 15W Fast Wireless Charging Station with Sleep-Friendly Adaptive Light Compatible with iPhone 14 13 12 Pro XR XS 8 Plus Samsung Galaxy S23 S22 S21 S20 Note 20 10 Google LG etc', 'Wireless Charger 2-Pack Qi-Certified 10W for iPhone Wireless Charger Pad COKOEYE, Max Fast Wireless Phone Charger for iPhone 14/14 Plu

In [7]:
import csv
header = ['product_name','product_price','product_ratings', 'product_ratings_num','product_link']
rows = zip(product_name,product_price,product_ratings,product_ratings_num,product_link)

filename = "output.csv"
with open(filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows)